In [1]:
# Classification: Iris (multiclass)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1) Data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# 2) Pipeline: Scale -> kNN
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

# 3) Hyperparameter grid
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 15],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2],                 # 1=Manhattan, 2=Euclidean
    "knn__algorithm": ["auto"]        # let sklearn decide; could try ['auto','kd_tree','ball_tree','brute']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1,
    verbose=0
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("CV best accuracy:", grid.best_score_)

# 4) Evaluate on test set
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print("\nTest set report:")
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Best params: {'knn__algorithm': 'auto', 'knn__n_neighbors': 3, 'knn__p': 2, 'knn__weights': 'uniform'}
CV best accuracy: 0.9731225296442687

Test set report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.81      1.00      0.90        13
           2       1.00      0.77      0.87        13

    accuracy                           0.92        38
   macro avg       0.94      0.92      0.92        38
weighted avg       0.94      0.92      0.92        38

Confusion matrix:
 [[12  0  0]
 [ 0 13  0]
 [ 0  3 10]]
